# Megatron XinLi QA fintune 模型 预测

## CD

定位到工作目录，根据具体情况决定哦，不一定是下面的命令

In [1]:
%cd ..

/home/Public/Megatron-LM


## Importings

In [2]:
import copy
import csv
import json
import os
import random
import sys
import time
from contextlib import closing
from types import SimpleNamespace

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

import mpu
from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer
from pretrain_gpt2 import get_masks_and_position_ids
from predict_gpt2 import initialize_distributed, prepare_tokenizer, set_random_seed, setup_model, get_token_stream

## Args

In [3]:
args = SimpleNamespace(
    # Model arguments
    num_layers=12,
    hidden_size=768,
    num_attention_heads=12,
    max_position_embeddings=1024,
    vocab_size=None,
    make_vocab_size_divisible_by=128,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    # Train/valid/test data arguments.
    seq_length=512,
    model_parallel_size=1,
    tokenizer_model_type='bert-large-uncased',
    tokenizer_type=SentencePieceTokenizer,
    tokenizer_path="./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model",
    cache_dir=None,
    # Training arguments.
    load='./checkpoints/xinliqa_hunyin/',
    seed=1234,
    checkpoint_activations=None,
    checkpoint_num_layers=1,
    finetune=None,
    no_load_optim=None,
    no_load_rng=None,
    resume_dataloader=None,
    fp16=True,
    hysteresis=2,
    loss_scale=None,
    loss_scale_window=1000,
    min_scale=1,
    distributed_backend='nccl',
    DDP_impl='local',
    local_rank=None,
    reset_position_ids=None,
    reset_attention_mask=None,
    eod_mask_loss=None, 
    # Text generate arguments.
    recompute=None,
    greedy=False,
    top_p=0.0,
    top_k=0,
    temperature=1.0,
    out_seq_length=128,
)

In [4]:
args.cuda = torch.cuda.is_available()
args.rank = int(os.getenv('RANK', '0'))
args.world_size = int(os.getenv("WORLD_SIZE", '1'))

if os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'):
    # We are using (OpenMPI) mpirun for launching distributed data parallel processes
    local_rank = int(os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'))
    local_size = int(os.getenv('OMPI_COMM_WORLD_LOCAL_SIZE'))

    # Possibly running with Slurm
    num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', '1'))
    nodeid = int(os.getenv('SLURM_NODEID', '0'))

    args.local_rank = local_rank
    args.rank = nodeid*local_size + local_rank
    args.world_size = num_nodes*local_size

args.model_parallel_size = min(args.model_parallel_size, args.world_size)
if args.rank == 0:
    print('using world size: {} and model-parallel size: {} '.format(
        args.world_size, args.model_parallel_size))

args.dynamic_loss_scale = False
if args.loss_scale is None:
    args.dynamic_loss_scale = True
    if args.rank == 0:
        print(' > using dynamic loss scaling')

# The args fp32_* or fp16_* meant to be active when the
# args fp16 is set. So the default behavior should all
# be false.
if not args.fp16:
    args.fp32_embedding = False
    args.fp32_tokentypes = False
    args.fp32_layernorm = False


using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling


## Init

In [5]:
%%time

# Disable CuDNN.
torch.backends.cudnn.enabled = False

# Pytorch distributed.
initialize_distributed(args)

# Random seeds for reproducability.
set_random_seed(args.seed)

# get the tokenizer
tokenizer = prepare_tokenizer(args)

# Model, optimizer, and learning rate.
model = setup_model(args)

args.device = torch.cuda.current_device()

# setting default batch size to 1
args.batch_size = 1

assert mpu.get_model_parallel_rank() == 0

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done
building GPT2 model ...
 > number of parameters on model parallel rank 0: 110516736
global rank 0 is loading checkpoint ./checkpoints/xinliqa_hunyin/iter_0055000/mp_rank_00/model_optim_rng.pt
  successfully loaded ./checkpoints/xinliqa_hunyin/iter_0055000/mp_rank_00/model_optim_rng.pt
CPU times: user 5.14 s, sys: 1.7 s, total: 6.84 s
Wall time: 6.85 s


## Inference functions

In [6]:
def infer_text_generative(contex_text, model, tokenizer):
    contex_text = contex_text.strip()
    context_tokens = tokenizer.EncodeAsIds(contex_text).tokenization
    context_length = len(context_tokens)

    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)
    
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        s = tokenizer.DecodeIds([ids[-1]])
        yield s


## Infer texts

In [7]:
input_texts = [
    (
        '最近晚上睡不着觉，但很困，总觉得不会睡觉？',
        '最近一段时间，总是晚上睡不着觉，但是却很困，没次睡觉心理都在不自觉的想着该如何去睡觉，每次想着想着就完全清醒了，失眠特别严重，感觉身体快要透支了，很无助，还连累家人一起跟着担心'
    ),
    (
        '大一，谈了将近一年的女友突然提出分手我该怎么办？',
        ' 某一天晚上她突然给我打电话说我不是她喜欢的那种类型，在电话中她多次提到自己的性格不好，说“我所看到的都是加了“滤镜”的效果，其实她并没有我想得那么好，今后不准备结婚”。（她以前谈过一次恋爱，对于那场恋爱她一直都铭记在心）接完电话我很迷茫和困惑，因为之前她愿意跟我分享她的生活中的不如意和困难，突然的转变让我措手不及。同时我也很自责，没有准确的表达出自己的意思和情感，在与她的对话中我常常感到无力和无助。我到底该怎么办？我真的很爱她，想和她度过一生。'
    ),
    (
        '男人出轨',
        '我的意思是，出轨者会问自己，我为什么要这么做，被背叛的伴侣会问，你为什么要这样对我?'
    ),
    (
        '我的奇葩婚姻，老公说为了孩子好就是不离婚？',
        '我的婚姻最近好像一塌糊涂，可是老公不离婚，我甚至有想过死？我是不是得了抑郁症？和老公从2017年开始就吵吵闹闹，一吵架就分居，分居一两个月老公可以对我和孩子不问不闻，到时间了他就自己又搬回来，但是回来也不和我沟通，就算是上次是他动手，他也不道歉，他特别喜欢冷战。现在和老公又发展到了无性，而且双方家庭也合不来，我和他家人不敢来往，他家人对我要求高，希望天天在家做饭带娃伺候老公，他现在和我父母也不来往，从来不回家吃饭，要等我父母走了他才回来，每天很晚回家，去哪从来不告诉我，我知道他有婚外性，但他從來不承認，还说我是神经病，我還被他傳播上了高危型的HPV，以后是死是活都不确定，我特别想要离婚，我感觉和他生活快要窒息了，他整天拉着个脸，要么对我不闻不问，要么一开口说话就是责怪的语气，但他就是不离婚，说是为了孩子好，还让我不要发神经，我真的不知道该怎么和他相处了。他性格很古怪'
    ),
    (
        '妻子在钱的问题上不坦诚，如何处理好家庭经济问题？',
        '和老婆因为意外怀孕结婚，两家谈婚论嫁时闹得不愉快，我家出了一套大户型的房子，一台豪华车（当然这些都是婚前财产），然后给了40万装修婚房，同时给女方10万彩礼。而女方一开始的态度是一分钱没有，后来迫于压力给了50万现金，我老婆名下既没有房也没有车。婚后的钱一直都是给我老婆管理，包括我的工资奖金还有结婚的份子钱等等。我老婆并没有把这些钱看做是家庭财产，就算其中已经掺入了很多我的钱，还是认为全部都是她自己的钱，我只是知道她的卡密码而已，但是卡在哪里，她花了多少钱，花在哪里了，有没有借钱给别人，我通通不知道，这让我很不安。况且她没有理财观念，我曾经三番五次跟她说了理财的重要性，她就只是沉默不语，也不知道是听不懂，还是装傻实际上自己去做了理财但是没把收入情况告诉我，我们多次吵架都是为了钱，我觉得夫妻两个在这个问题上不能坦诚是个大问题，我现在很想经济分开，但是不知道怎么做才比较妥当'
    )
]


In [8]:
# args.recompute=True
args.top_p=0.0
args.top_k=0
args.temperature=1.0

n_gen = 3

for title, text in input_texts:
    txt = f'{title}<sep>{text}<|endoftext|>'
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    print(title)
    print(text)
    print()
    for i in range(n_gen):
        args.temperature=random.gauss(0.95, 0.05)
        print(f'{i}: ', end='')
        s_pred = ''
        for s in infer_text_generative(txt, model, tokenizer):
            print(s, end='')
        print(os.linesep)
    print()
    print('=' * 100)
    print()
    

最近晚上睡不着觉，但很困，总觉得不会睡觉？
最近一段时间，总是晚上睡不着觉，但是却很困，没次睡觉心理都在不自觉的想着该如何去睡觉，每次想着想着就完全清醒了，失眠特别严重，感觉身体快要透支了，很无助，还连累家人一起跟着担心

0: <bos>你好,找你倾诉一下你的情绪,打击与你的情结,还有爱浪子,还有爱。虽然婚姻不是简单地年少,但你也建议你用婚姻的智慧去争取陪伴和繁华。 <eos>

1: <bos>我能感受到父母的话,我们养育了三年的狗,可是没有给带走的人,活在另一个极端,近乎的朋友,后来姐姐开车去住,由于地下室太远了,很多朋友的小姑娘是塞保心。2.没有人教我舔,天天在公司上班老和同事玩,况且他不想回家就抽烟喝酒,一周也不会带人出去玩,出去找点酒喝完酒,事后问人都没不可以再去我家,可是只要他们就会逼我,说我是那种人,而我,情感上很难受,有时候甚至想死的心都有了。我不喜欢听

2: <bos>讲下那就是成长过程中的一种分离焦虑,然而创伤后遗症是躯体、经历..虚构的并不是对象,而是掌握了内心敏感时间和精力,继续或长了会影响自信心,自然不会有稳定的,你能选择来这里求助,就非常的不简单。咨询是一个过程,我需要了解更多的情况,才能帮你分析解决问题。我帮助过很多和你一样困扰的人,相信也可以帮到你。可以多说一些你的情况吗? <eos>



大一，谈了将近一年的女友突然提出分手我该怎么办？
 某一天晚上她突然给我打电话说我不是她喜欢的那种类型，在电话中她多次提到自己的性格不好，说“我所看到的都是加了“滤镜”的效果，其实她并没有我想得那么好，今后不准备结婚”。（她以前谈过一次恋爱，对于那场恋爱她一直都铭记在心）接完电话我很迷茫和困惑，因为之前她愿意跟我分享她的生活中的不如意和困难，突然的转变让我措手不及。同时我也很自责，没有准确的表达出自己的意思和情感，在与她的对话中我常常感到无力和无助。我到底该怎么办？我真的很爱她，想和她度过一生。

0: <bos>我也是一样,我特喜欢她,可她不喜欢的人在一起,要感觉很开心,可是她很介意她,不想看见我的留言,虽然我知道她不好意思帮忙,希望她全心全意的为她做爱侣,能做朋友的最好,可是,我的内心深处还是有些悲伤,我现在特别特别想和她在一起,虽然我很难受,但是她总是为我想不开,希望有人可以帮帮我。<|endoftext|><bos>你好。你们在一起多